In [1]:

import os
import pandas as pd
from sklearn.cluster import KMeans
import numpy as np
# 设置显示小数点后两位
pd.options.display.float_format = '{:.2f}'.format
# 设置显示最大行数
pd.options.display.max_rows = 100

In [2]:
# 数据分段
def natural_breaks_classification(data_column, num_classes=5):
    """
    使用自然间断法(Jenks)对数据进行分类，返回分段边界和标签
    
    参数:
    data_column: pandas.Series - 需要分段的数据列
    num_classes: int - 分段数量，默认为5
    
    返回:
    tuple - (bins, labels) 可直接用于pd.cut()函数
    """

    # 处理空值
    values = data_column.dropna().values.reshape(-1, 1)
    
    if len(values) <= num_classes:
        # 数据点太少，使用等间距分段
        min_val = data_column.min()
        max_val = data_column.max()
        step = (max_val - min_val) / num_classes
        bins = [min_val + i * step for i in range(num_classes+1)]
        bins[0] = 0  # 确保最小值是0
        bins[-1] = float('inf')  # 确保最大值是无穷大
    else:
        # 使用KMeans进行自然间断分类
        kmeans = KMeans(n_clusters=num_classes, random_state=42)
        kmeans.fit(values)
        
        # 获取聚类中心并排序
        centers = sorted(kmeans.cluster_centers_.flatten())
        
        # 创建边界值(bins）- 使用聚类中心的中点作为分割点
        bins = [0]  # 起始点为0
        bins.extend([(centers[i] + centers[i+1])/2 for i in range(len(centers)-1)])
        bins.append(float('inf'))  # 结束点为无穷大
    
    # 四舍五入边界值以便更好理解
    rounded_bins = [0]
    for b in bins[1:-1]:
        rounded_bins.append(int(round(b)))
    rounded_bins.append(float('inf'))
    bins = rounded_bins
    
    # 创建标签
    labels = []
    for i in range(len(bins)-1):
        if i == 0:
            labels.append(f'<{bins[i+1]}')
        elif i == len(bins)-2:
            labels.append(f'>{bins[i]}')
        else:
            labels.append(f'{bins[i]}-{bins[i+1]}')
    
    return bins, labels

def table_7_6(df, level_columns='综合质量等级', area_columns='project_Area', capacity_columns='annual_production'):
    # 复制输入数据，避免修改原始 DataFrame
    calc_df = df.copy()
    # 将面积从平方米转换为亩（1 亩 ≈ 666.67 平方米，转换因子为 0.0015）
    calc_df[area_columns] = calc_df[area_columns] * 0.0015
    total_area = calc_df[area_columns].sum()
    
    # 按质量等级分组，计算面积总和和年产能均值
    quality_stats = calc_df.groupby(level_columns).agg({
        area_columns: 'sum',
        capacity_columns: 'mean'  # 年产能假设为每亩的平均值
    }).reset_index()
    quality_stats['比例/%'] = (quality_stats[area_columns] / total_area * 100).round(2)
    quality_stats.columns = ['等级', '面积/亩', '年产能(kg/亩)', '比例/%']
    
    # 定义分级映射
    quality_map = {
        '一等': '高等级耕地', '二等': '高等级耕地', '三等': '高等级耕地',
        '四等': '中等级耕地', '五等': '中等级耕地', '六等': '中等级耕地',
        '七等': '低等级耕地', '八等': '低等级耕地', '九等': '低等级耕地', '十等': '低等级耕地'
    }
    quality_stats['分级'] = quality_stats['等级'].map(quality_map)
    
    # 定义排序顺序
    grade_order = {
        '高等级耕地': 1, '中等级耕地': 2, '低等级耕地': 3
    }
    level_order = {
        '一等': 1, '二等': 2, '三等': 3, '四等': 4, '五等': 5,
        '六等': 6, '七等': 7, '八等': 8, '九等': 9, '十等': 10
    }
    
    # 添加辅助列用于排序
    quality_stats['分级排序'] = quality_stats['分级'].map(grade_order)
    quality_stats['等级排序'] = quality_stats['等级'].map(level_order)
    
    # 按分级和等级排序
    quality_stats = quality_stats.sort_values(by=['分级排序', '等级排序'])
    
    # 计算加权平均年产能（用于总计行）
    if total_area > 0:
        weighted_avg_capacity = (quality_stats['面积/亩'] * quality_stats['年产能(kg/亩)']).sum() / total_area
    else:
        weighted_avg_capacity = 0
    
    # 创建总计行
    total_row = pd.DataFrame([{
        '分级': '总计',
        '等级': '总计',
        '面积/亩': total_area,
        '比例/%': 100,
        '年产能(kg/亩)': weighted_avg_capacity
    }])
    
    # 将总计行添加到结果中
    quality_stats = pd.concat(
        [quality_stats[['分级', '等级', '面积/亩', '比例/%', '年产能(kg/亩)']], total_row],
        ignore_index=True
    )
    
    # 返回最终结果
    return quality_stats


# 表7.7 各乡镇耕地质量等级情况
def table_7_7(df, township_columns='XZQMC', level_columns='综合质量等级', area_columns='project_Area'):
    calc_df = df.copy()
    # 将面积单位(平方米)换算为亩
    calc_df[area_columns] = calc_df[area_columns] * 0.0015
    
    # 定义高中低等级耕地的映射
    high_grades = ['一等', '二等', '三等']
    medium_grades = ['四等', '五等', '六等']
    low_grades = ['七等', '八等', '九等', '十等']
    
    # 创建分级列
    calc_df['分级'] = calc_df[level_columns].apply(lambda x: 
                                              '高等级耕地' if x in high_grades else
                                              '中等级耕地' if x in medium_grades else
                                              '低等级耕地' if x in low_grades else None)
    
    # 按乡镇和分级分组，计算面积总和
    township_grade_stats = calc_df.groupby([township_columns, '分级'])[area_columns].sum().reset_index()
    
    # 透视表转换为宽格式
    pivot_table = township_grade_stats.pivot(index=township_columns, 
                                            columns='分级', 
                                            values=area_columns).reset_index()
    
    # 确保所有等级列都存在，如果不存在则填充0
    for grade in ['高等级耕地', '中等级耕地', '低等级耕地']:
        if grade not in pivot_table.columns:
            pivot_table[grade] = 0
    
    # 重命名列
    result = pivot_table.rename(columns={
        township_columns: '乡镇',
        '高等级耕地': '高等级耕地（亩）',
        '中等级耕地': '中等级耕地（亩）',
        '低等级耕地': '低等级耕地（亩）'
    })
    
    # 计算合计面积
    result['合计（亩）'] = result[['高等级耕地（亩）', '中等级耕地（亩）', '低等级耕地（亩）']].sum(axis=1).round(2)
    
    # 计算各等级占比
    for grade in ['高等级耕地', '中等级耕地', '低等级耕地']:
        # 安全计算占比，防止除零错误
        result[f'{grade}_占比'] = result.apply(
            lambda row: round(row[f'{grade}（亩）'] / row['合计（亩）'] * 100, 2) 
            if row['合计（亩）'] > 0 else 0.0, 
            axis=1
        )
    
    # 添加总计行
    total_row = pd.DataFrame({
        '乡镇': ['合计'],
        '高等级耕地（亩）': [result['高等级耕地（亩）'].sum().round(2)],
        '中等级耕地（亩）': [result['中等级耕地（亩）'].sum().round(2)],
        '低等级耕地（亩）': [result['低等级耕地（亩）'].sum().round(2)]
    })
    
    # 计算总计行的合计面积
    total_row['合计（亩）'] = total_row[['高等级耕地（亩）', '中等级耕地（亩）', '低等级耕地（亩）']].sum(axis=1).round(2)
    
    # 计算总计行的占比
    total_area = total_row['合计（亩）'].iloc[0]
    for grade in ['高等级耕地', '中等级耕地', '低等级耕地']:
        if total_area > 0:
            total_row[f'{grade}_占比'] = [round(total_row[f'{grade}（亩）'].iloc[0] / total_area * 100, 2)]
        else:
            total_row[f'{grade}_占比'] = [0.0]
    
    # 合并结果
    result = pd.concat([result, total_row], ignore_index=True)
    
    # 确保列顺序与图表一致
    column_order = ['乡镇', '高等级耕地（亩）', '高等级耕地_占比', '中等级耕地（亩）', '中等级耕地_占比', 
                    '低等级耕地（亩）', '低等级耕地_占比', '合计（亩）']
    result = result[column_order]
    
    # 确保所有数值列都是数值类型
    for col in result.columns:
        if col != '乡镇':
            result[col] = pd.to_numeric(result[col], errors='coerce').fillna(0)
    
    return result


# 表7.8 土壤类型耕地质量等级情况
def table_7_8(df, area_columns='project_Area', level_columns='综合质量等级', 
              type_column='Class1_tl', sub_column='Class1_yl', cate_column='Class1_ts', spe_column='Class1'):
    calc_df = df.copy()
    # 将面积单位(平方米)换算为亩
    calc_df[area_columns] = calc_df[area_columns] * 0.0015
    
    # 定义等级分类
    high_grades = ['一等', '二等', '三等']
    medium_grades = ['四等', '五等', '六等']
    low_grades = ['七等', '八等', '九等', '十等']
    
    # 添加等级分类列
    def classify_grade(grade):
        if grade in high_grades:
            return '高等级耕地'
        elif grade in medium_grades:
            return '中等级耕地'
        else:
            return '低等级耕地'
    
    calc_df['等级分类'] = calc_df[level_columns].apply(classify_grade)
    
    # 按土壤类型和等级分类分组统计面积
    soil_stats = calc_df.groupby([type_column, sub_column, cate_column, spe_column, '等级分类'])[area_columns].sum().unstack(fill_value=0)
    
    # 确保三个等级列都存在
    for col in ['高等级耕地', '中等级耕地', '低等级耕地']:
        if col not in soil_stats.columns:
            soil_stats[col] = 0
    
    soil_stats['合计'] = soil_stats.sum(axis=1)
    
    # 计算各等级占比
    soil_stats['高等级耕地_占比'] = (soil_stats['高等级耕地'] / soil_stats['合计'] * 100).round(2)
    soil_stats['中等级耕地_占比'] = (soil_stats['中等级耕地'] / soil_stats['合计'] * 100).round(2)
    soil_stats['低等级耕地_占比'] = (soil_stats['低等级耕地'] / soil_stats['合计'] * 100).round(2)
    
    # 计算总计行数据
    total_high = soil_stats['高等级耕地'].sum()
    total_medium = soil_stats['中等级耕地'].sum()
    total_low = soil_stats['低等级耕地'].sum()
    total_all = total_high + total_medium + total_low
    
    # 重置索引并重命名列
    result = soil_stats.reset_index()
    result = result.rename(columns={
        type_column: '土类', sub_column: '亚类', 
        cate_column: '土属', spe_column: '土种',
        '高等级耕地': '面积(亩)_高', '高等级耕地_占比': '占比(%)_高',
        '中等级耕地': '面积(亩)_中', '中等级耕地_占比': '占比(%)_中',
        '低等级耕地': '面积(亩)_低', '低等级耕地_占比': '占比(%)_低'
    })
    
    # 添加总计行
    total_row = pd.DataFrame({
        '土类': ['合计'], '亚类': [''], '土属': [''], '土种': [''],
        '面积(亩)_高': [total_high], 
        '占比(%)_高': [(total_high / total_all * 100).round(2) if total_all > 0 else 0],
        '面积(亩)_中': [total_medium], 
        '占比(%)_中': [(total_medium / total_all * 100).round(2) if total_all > 0 else 0],
        '面积(亩)_低': [total_low], 
        '占比(%)_低': [(total_low / total_all * 100).round(2) if total_all > 0 else 0],
        '合计': [total_all]
    })
    
    result = pd.concat([result, total_row], ignore_index=True)
    
    # 整理最终列顺序
    columns_order = ['土类', '亚类', '土属', '土种', 
                     '面积(亩)_高', '占比(%)_高', 
                     '面积(亩)_中', '占比(%)_中', 
                     '面积(亩)_低', '占比(%)_低', 
                     '合计']
    
    return result[columns_order]

# 表7.9 一等耕地分布特征统计表
def table_7_9_distribution(df,area_columns='project_Area',level_columns='综合质量等级',township_columns='XZQMC',level_value='一等'):
    calc_df = df.copy()
    calc_df[area_columns] = calc_df[area_columns] * 0.0015
    first_grade = calc_df[calc_df[level_columns] == level_value]
    township_stats = first_grade.groupby(township_columns)[area_columns].sum()
    total_area = township_stats.sum()
    result = pd.DataFrame({
        '面积（亩）': township_stats,
        '比例（%）': (township_stats / total_area * 100).round(2)
    }).reset_index().rename(columns={township_columns: '乡镇'})
    # 添加总计行
    total_row = pd.DataFrame({
        '乡镇': ['合计'],
        '面积（亩）': result['面积（亩）'].sum(),
        '比例（%）': result['比例（%）'].sum()
    })
    result = pd.concat([result, total_row], ignore_index=True)
    return result

# 表7.9 一等耕地性状特征统计表
def table_7_9_properties(df,area_columns='project_Area',level_columns='综合质量等级',type_column='Class1_tl',sub_column='Class1_yl',cate_column='Class1_ts',spe_column='Class1',level_value='一等',capacity_columns='annual_production'):
    calc_df = df.copy()
    calc_df[area_columns] = calc_df[area_columns] * 0.0015
    first_grade = calc_df[calc_df[level_columns] == level_value]
    soil_stats = first_grade.groupby([type_column, sub_column, cate_column, spe_column]).agg({
        area_columns: 'sum',
        capacity_columns: 'mean'
    }).reset_index()
    total_area = soil_stats[area_columns].sum()
    soil_stats['比例（%）'] = (soil_stats[area_columns] / total_area * 100).round(2)
    soil_stats.columns = ['土类', '亚类', '土属', '土种', '面积（亩）', '年产能（kg/亩）', '比例（%）']
    # 添加总计行
    total_row = pd.DataFrame({
        '土类': ['合计'],
        '亚类': [''],
        '土属': [''],
        '土种': [''],
        '面积（亩）': soil_stats['面积（亩）'].sum(), # 总和
        '年产能（kg/亩）': soil_stats['年产能（kg/亩）'].sum(), # 总和
        '比例（%）': (soil_stats['面积（亩）'].sum() / total_area * 100).round(2)
    })
    result = pd.concat([soil_stats, total_row], ignore_index=True)
    return result

# 各等级耕地地形部位对比
# 各等级耕地地形部位对比
def table_terrain(df, area_columns='project_Area', level_columns='综合质量等级', terrain_column='地形部位'):
    # 将面积单位(平方米)换算为亩
    calc_df = df.copy()
    calc_df[area_columns] = calc_df[area_columns] * 0.0015
    
    # 按地形部位和等级分组统计面积
    terrain_stats = calc_df.groupby([terrain_column, level_columns])[area_columns].sum().unstack(fill_value=0)
    
    # 确保所有等级列存在（一等到十等）
    for grade in ['一等', '二等', '三等', '四等', '五等', '六等', '七等', '八等', '九等', '十等']:
        if grade not in terrain_stats.columns:
            terrain_stats[grade] = 0
    
    # 添加合计行
    terrain_stats.loc['合计'] = terrain_stats.sum()
    
    # 按照图片中的顺序排列地形部位
    terrain_order = ['山地坡上', '山地坡中', '山地坡下', '合计']
    
    # 重新整理结果DataFrame以匹配所需格式
    result_df = pd.DataFrame()
    result_df['地形部位'] = terrain_order
    
    # 设置结果DataFrame的索引以便后续操作
    result_df = result_df.set_index('地形部位')
    
    # 对于每个等级，添加面积和比例列
    for grade in ['一等', '二等', '三等', '四等', '五等', '六等', '七等', '八等', '九等', '十等']:
        # 为不存在的地形部位填充0
        for terrain in terrain_order:
            if terrain != '合计' and terrain not in terrain_stats.index:
                terrain_stats.loc[terrain, grade] = 0
        
        # 计算每个地形部位在该等级中的比例
        total_grade_area = terrain_stats.loc['合计', grade] if '合计' in terrain_stats.index else terrain_stats[grade].sum()
        
        if total_grade_area > 0:
            for terrain in terrain_order:
                if terrain != '合计':
                    # 如果地形部位在统计数据中不存在，则面积为0
                    area = terrain_stats.loc[terrain, grade] if terrain in terrain_stats.index else 0
                    ratio = (area / total_grade_area * 100).round(2) if total_grade_area > 0 else 0
                else:
                    # 合计行的数据
                    area = total_grade_area
                    ratio = 100.0
                
                # 添加到结果DataFrame
                result_df.loc[terrain, f'{grade}_面积/亩'] = area
                result_df.loc[terrain, f'{grade}_比例/%'] = ratio
    
    # 重置索引并返回结果
    result_df = result_df.reset_index()
    return result_df

# 各等级耕地有效土层厚度对比
def table_thickness(df, area_columns='project_Area', level_columns='综合质量等级', thickness_column='有效土层厚度'):
    # 将面积单位(平方米)换算为亩
    calc_df = df.copy()
    calc_df[area_columns] = calc_df[area_columns] * 0.0015
    
    # 将有效土层厚度分段
    bins = [0, 30, 50, 80, float('inf')]
    labels = ['<30', '30-50', '50-80', '≥80']
    calc_df['厚度区间'] = pd.cut(calc_df[thickness_column], bins=bins, labels=labels, right=False)
    
    # 按厚度区间和等级分组统计面积
    thickness_stats = calc_df.groupby(['厚度区间', level_columns])[area_columns].sum().unstack(fill_value=0)
    
    # 确保所有等级列存在（一等到十等）
    for grade in ['一等', '二等', '三等', '四等', '五等', '六等', '七等', '八等', '九等', '十等']:
        if grade not in thickness_stats.columns:
            thickness_stats[grade] = 0
    
    # 添加合计行
    thickness_stats.loc['合计'] = thickness_stats.sum()
    
    # 按照图片中的顺序排列厚度区间
    thickness_order = ['≥80', '50-80', '30-50', '<30', '合计']
    
    # 重新整理结果DataFrame以匹配所需格式
    result_df = pd.DataFrame()
    result_df['有效土层厚度（cm）'] = thickness_order
    
    # 设置结果DataFrame的索引以便后续操作
    result_df = result_df.set_index('有效土层厚度（cm）')
    
    # 对于每个等级，添加面积和比例列
    for grade in ['一等', '二等', '三等', '四等', '五等', '六等', '七等', '八等', '九等', '十等']:
        # 为不存在的厚度区间填充0
        for thickness in thickness_order:
            if thickness != '合计' and thickness not in thickness_stats.index:
                thickness_stats.loc[thickness, grade] = 0
        
        # 计算每个厚度区间在该等级中的比例
        total_grade_area = thickness_stats.loc['合计', grade] if '合计' in thickness_stats.index else thickness_stats[grade].sum()
        
        if total_grade_area > 0:
            for thickness in thickness_order:
                if thickness != '合计':
                    # 如果厚度区间在统计数据中不存在，则面积为0
                    thickness_key = thickness
                    if thickness == '≥80':
                        thickness_key = '≥80'
                    area = thickness_stats.loc[thickness_key, grade] if thickness_key in thickness_stats.index else 0
                    ratio = (area / total_grade_area * 100).round(2) if total_grade_area > 0 else 0
                else:
                    # 合计行的数据
                    area = total_grade_area
                    ratio = 100.0
                
                # 添加到结果DataFrame
                result_df.loc[thickness, f'{grade}_面积/亩'] = area
                result_df.loc[thickness, f'{grade}_比例/%'] = ratio
    
    # 重置索引并返回结果
    result_df = result_df.reset_index()
    return result_df

# 各等级耕地质地构型对比
def table_texture_config(df, area_columns='project_Area', level_columns='综合质量等级', texture_column='质地构型'):
    # 将面积单位(平方米)换算为亩
    calc_df = df.copy()
    calc_df[area_columns] = calc_df[area_columns] * 0.0015
    
    # 按质地构型和等级分组统计面积
    texture_stats = calc_df.groupby([texture_column, level_columns])[area_columns].sum().unstack(fill_value=0)
    
    # 确保所有等级列存在（一等到十等）
    for grade in ['一等', '二等', '三等', '四等', '五等', '六等', '七等', '八等', '九等', '十等']:
        if grade not in texture_stats.columns:
            texture_stats[grade] = 0
    
    # 添加合计行
    texture_stats.loc['合计'] = texture_stats.sum()
    
    # 按照图片中的顺序排列质地构型
    texture_order = ['上松下紧型', '海绵型', '夹层型', '紧实型', '上紧下松型', '松散型', '薄层型', '合计']
    
    # 重新整理结果DataFrame以匹配所需格式
    result_df = pd.DataFrame()
    result_df['质地构型'] = texture_order
    
    # 设置结果DataFrame的索引以便后续操作
    result_df = result_df.set_index('质地构型')
    
    # 对于每个等级，添加面积和比例列
    for grade in ['一等', '二等', '三等', '四等', '五等', '六等', '七等', '八等', '九等', '十等']:
        # 为不存在的质地构型填充0
        for texture in texture_order:
            if texture != '合计' and texture not in texture_stats.index:
                texture_stats.loc[texture, grade] = 0
        
        # 计算每个质地构型在该等级中的比例
        total_grade_area = texture_stats.loc['合计', grade] if '合计' in texture_stats.index else texture_stats[grade].sum()
        
        if total_grade_area > 0:
            for texture in texture_order:
                if texture != '合计':
                    # 如果质地构型在统计数据中不存在，则面积为0
                    area = texture_stats.loc[texture, grade] if texture in texture_stats.index else 0
                    ratio = (area / total_grade_area * 100).round(2) if total_grade_area > 0 else 0
                else:
                    # 合计行的数据
                    area = total_grade_area
                    ratio = 100.0
                
                # 添加到结果DataFrame
                result_df.loc[texture, f'{grade}_面积/亩'] = area
                result_df.loc[texture, f'{grade}_比例/%'] = ratio
    
    # 重置索引并返回结果
    result_df = result_df.reset_index()
    return result_df

# 各等级耕地海拔高度对比
def table_elevation(df, area_columns='project_Area', level_columns='综合质量等级', elevation_column='海拔高度'):
    # 将面积单位(平方米)换算为亩
    calc_df = df.copy()
    calc_df[area_columns] = calc_df[area_columns] * 0.0015
    
    # 将海拔高度分段
    # bins = [0, 850, 1000, 1150, 1300, float('inf')]
    # labels = ['<850', '850-1000', '1000-1150', '1150-1300', '>1300']
    bins, labels = natural_breaks_classification(calc_df[elevation_column])
    calc_df['海拔区间'] = pd.cut(calc_df[elevation_column], bins=bins, labels=labels, right=False)
    
    # 按海拔区间和等级分组统计面积
    elevation_stats = calc_df.groupby(['海拔区间', level_columns])[area_columns].sum().unstack(fill_value=0)
    
    # 确保所有等级列存在（一等到十等）
    for grade in ['一等', '二等', '三等', '四等', '五等', '六等', '七等', '八等', '九等', '十等']:
        if grade not in elevation_stats.columns:
            elevation_stats[grade] = 0
    
    # 添加合计行
    elevation_stats.loc['合计'] = elevation_stats.sum()
    
    # 按照图片中的顺序排列海拔区间
    # elevation_order = ['<850', '850-1000', '1000-1150', '1150-1300', '>1300', '合计']
    elevation_order = labels+['合计']
    
    # 重新整理结果DataFrame以匹配所需格式
    result_df = pd.DataFrame()
    result_df['海拔高度（m）'] = elevation_order
    
    # 设置结果DataFrame的索引以便后续操作
    result_df = result_df.set_index('海拔高度（m）')
    
    # 对于每个等级，添加面积和比例列
    for grade in ['一等', '二等', '三等', '四等', '五等', '六等', '七等', '八等', '九等', '十等']:
        # 为不存在的海拔区间填充0
        for elevation in elevation_order:
            if elevation != '合计' and elevation not in elevation_stats.index:
                elevation_stats.loc[elevation, grade] = 0
        
        # 计算每个海拔区间在该等级中的比例
        total_grade_area = elevation_stats.loc['合计', grade] if '合计' in elevation_stats.index else elevation_stats[grade].sum()
        
        if total_grade_area > 0:
            for elevation in elevation_order:
                if elevation != '合计':
                    # 如果海拔区间在统计数据中不存在，则面积为0
                    area = elevation_stats.loc[elevation, grade] if elevation in elevation_stats.index else 0
                    ratio = (area / total_grade_area * 100).round(2) if total_grade_area > 0 else 0
                else:
                    # 合计行的数据
                    area = total_grade_area
                    ratio = 100.0
                
                # 添加到结果DataFrame
                result_df.loc[elevation, f'{grade}_面积/亩'] = area
                result_df.loc[elevation, f'{grade}_比例/%'] = ratio
    
    # 重置索引并返回结果
    result_df = result_df.reset_index()
    return result_df

# 各等级耕地耕层厚度对比
def table_layer_thickness(df, area_columns='project_Area', level_columns='综合质量等级', thickness_column='耕层厚度'):
    # 将面积单位(平方米)换算为亩
    calc_df = df.copy()
    calc_df[area_columns] = calc_df[area_columns] * 0.0015
    
    # 将耕层厚度分段
    bins = [0, 10, 15, 20, float('inf')]
    labels = ['<10', '10-15', '15-20', '>20']
    calc_df['耕层厚度区间'] = pd.cut(calc_df[thickness_column], bins=bins, labels=labels, right=False)
    
    # 按耕层厚度区间和等级分组统计面积
    thickness_stats = calc_df.groupby(['耕层厚度区间', level_columns])[area_columns].sum().unstack(fill_value=0)
    
    # 确保所有等级列存在（一等到十等）
    for grade in ['一等', '二等', '三等', '四等', '五等', '六等', '七等', '八等', '九等', '十等']:
        if grade not in thickness_stats.columns:
            thickness_stats[grade] = 0
    
    # 添加合计行
    thickness_stats.loc['合计'] = thickness_stats.sum()
    
    # 按照图片中的顺序排列耕层厚度区间
    thickness_order = ['<10', '10-15', '15-20', '>20', '合计']
    
    # 重新整理结果DataFrame以匹配所需格式
    result_df = pd.DataFrame()
    result_df['耕层厚度（cm）'] = thickness_order
    
    # 设置结果DataFrame的索引以便后续操作
    result_df = result_df.set_index('耕层厚度（cm）')
    
    # 对于每个等级，添加面积和比例列
    for grade in ['一等', '二等', '三等', '四等', '五等', '六等', '七等', '八等', '九等', '十等']:
        # 为不存在的耕层厚度区间填充0
        for thickness in thickness_order:
            if thickness != '合计' and thickness not in thickness_stats.index:
                thickness_stats.loc[thickness, grade] = 0
        
        # 计算每个耕层厚度区间在该等级中的比例
        total_grade_area = thickness_stats.loc['合计', grade] if '合计' in thickness_stats.index else thickness_stats[grade].sum()
        
        if total_grade_area > 0:
            for thickness in thickness_order:
                if thickness != '合计':
                    # 如果耕层厚度区间在统计数据中不存在，则面积为0
                    area = thickness_stats.loc[thickness, grade] if thickness in thickness_stats.index else 0
                    ratio = (area / total_grade_area * 100).round(2) if total_grade_area > 0 else 0
                else:
                    # 合计行的数据
                    area = total_grade_area
                    ratio = 100.0
                
                # 添加到结果DataFrame
                result_df.loc[thickness, f'{grade}_面积/亩'] = area
                result_df.loc[thickness, f'{grade}_比例/%'] = ratio
    
    # 重置索引并返回结果
    result_df = result_df.reset_index()
    return result_df

# 各等级耕地耕层质地对比
def table_layer_texture(df, area_columns='project_Area', level_columns='综合质量等级', texture_column='耕层质地'):
    # 将面积单位(平方米)换算为亩
    calc_df = df.copy()
    calc_df[area_columns] = calc_df[area_columns] * 0.0015
    
    # 按耕层质地和等级分组统计面积
    texture_stats = calc_df.groupby([texture_column, level_columns])[area_columns].sum().unstack(fill_value=0)
    
    # 确保所有等级列存在（一等到十等）
    for grade in ['一等', '二等', '三等', '四等', '五等', '六等', '七等', '八等', '九等', '十等']:
        if grade not in texture_stats.columns:
            texture_stats[grade] = 0
    
    # 添加合计行
    texture_stats.loc['合计'] = texture_stats.sum()
    
    # 按照图片中的顺序排列耕层质地类型
    texture_order = [
        '壤土', 
        '粉(砂)质壤土', 
        '粘壤土', 
        '粉(砂)质黏壤土', 
        '粉(砂)质黏土',
        '砂质黏壤土',
        '砂质壤土',
        '壤质黏土',
        '黏土',
        '砂质黏土',
        '重黏土',
        '砂土及壤质砂土',
        '合计'
    ]
    
    # 重新整理结果DataFrame以匹配所需格式
    result_df = pd.DataFrame()
    result_df['耕层质地'] = texture_order
    
    # 设置结果DataFrame的索引以便后续操作
    result_df = result_df.set_index('耕层质地')
    
    # 对于每个等级，添加面积和比例列
    for grade in ['一等', '二等', '三等', '四等', '五等', '六等', '七等', '八等', '九等', '十等']:
        # 为不存在的耕层质地类型填充0
        for texture in texture_order:
            if texture != '合计' and texture not in texture_stats.index:
                texture_stats.loc[texture, grade] = 0
        
        # 计算每个耕层质地类型在该等级中的比例
        total_grade_area = texture_stats.loc['合计', grade] if '合计' in texture_stats.index else texture_stats[grade].sum()
        
        if total_grade_area > 0:
            for texture in texture_order:
                if texture != '合计':
                    # 如果耕层质地类型在统计数据中不存在，则面积为0
                    area = texture_stats.loc[texture, grade] if texture in texture_stats.index else 0
                    ratio = (area / total_grade_area * 100).round(2) if total_grade_area > 0 else 0
                else:
                    # 合计行的数据
                    area = total_grade_area
                    ratio = 100.0
                
                # 添加到结果DataFrame
                result_df.loc[texture, f'{grade}_面积/亩'] = area
                result_df.loc[texture, f'{grade}_比例/%'] = ratio
    
    # 重置索引并返回结果
    result_df = result_df.reset_index()
    return result_df

# 各等级耕地土壤容重对比
def table_bulk_density(df, area_columns='project_Area', level_columns='综合质量等级', density_column='土壤容重'):
    # 将面积单位(平方米)换算为亩
    calc_df = df.copy()
    calc_df[area_columns] = calc_df[area_columns] * 0.0015
    
    # 将土壤容重分段
    bins = [0, 1.00, 1.25, 1.35, 1.45, 1.55, float('inf')]
    labels = ['<1.00', '1.00-1.25', '1.25-1.35', '1.35-1.45', '1.45-1.55', '>1.55']
    calc_df['容重区间'] = pd.cut(calc_df[density_column], bins=bins, labels=labels, right=False)
    
    # 按容重区间和等级分组统计面积
    density_stats = calc_df.groupby(['容重区间', level_columns])[area_columns].sum().unstack(fill_value=0)
    
    # 确保所有等级列存在（一等到十等）
    for grade in ['一等', '二等', '三等', '四等', '五等', '六等', '七等', '八等', '九等', '十等']:
        if grade not in density_stats.columns:
            density_stats[grade] = 0
    
    # 添加合计行
    density_stats.loc['合计'] = density_stats.sum()
    
    # 按照图片中的顺序排列容重区间
    density_order = ['<1.00', '1.00-1.25', '1.25-1.35', '1.35-1.45', '1.45-1.55', '>1.55', '合计']
    
    # 重新整理结果DataFrame以匹配所需格式
    result_df = pd.DataFrame()
    result_df['土壤容重（g/cm³）'] = density_order
    
    # 设置结果DataFrame的索引以便后续操作
    result_df = result_df.set_index('土壤容重（g/cm³）')
    
    # 对于每个等级，添加面积和比例列
    for grade in ['一等', '二等', '三等', '四等', '五等', '六等', '七等', '八等', '九等', '十等']:
        # 为不存在的容重区间填充0
        for density in density_order:
            if density != '合计' and density not in density_stats.index:
                density_stats.loc[density, grade] = 0
        
        # 计算每个容重区间在该等级中的比例
        total_grade_area = density_stats.loc['合计', grade] if '合计' in density_stats.index else density_stats[grade].sum()
        
        if total_grade_area > 0:
            for density in density_order:
                if density != '合计':
                    # 如果容重区间在统计数据中不存在，则面积为0
                    area = density_stats.loc[density, grade] if density in density_stats.index else 0
                    ratio = (area / total_grade_area * 100).round(2) if total_grade_area > 0 else 0
                else:
                    # 合计行的数据
                    area = total_grade_area
                    ratio = 100.0
                
                # 添加到结果DataFrame
                result_df.loc[density, f'{grade}_面积/亩'] = area
                result_df.loc[density, f'{grade}_比例/%'] = ratio
    
    # 重置索引并返回结果
    result_df = result_df.reset_index()
    return result_df

# 各等级耕地土壤酸碱度对比
def table_ph(df, area_columns='project_Area', level_columns='综合质量等级', density_column='酸碱度'):
    # 将面积单位(平方米)换算为亩
    calc_df = df.copy()
    calc_df[area_columns] = calc_df[area_columns] * 0.0015
    
    # 将酸碱度分段
    bins = [0, 4.50, 5.50, 6.50, 7.50, float('inf')]
    labels = ['<4.50', '4.50-5.50', '5.50-6.50', '6.50-7.50', '>7.50']
    calc_df['酸碱度区间'] = pd.cut(calc_df[density_column], bins=bins, labels=labels, right=False)
    
    # 按酸碱度区间和等级分组统计面积
    density_stats = calc_df.groupby(['酸碱度区间', level_columns])[area_columns].sum().unstack(fill_value=0)
    
    # 确保所有等级列存在（一等到十等）
    for grade in ['一等', '二等', '三等', '四等', '五等', '六等', '七等', '八等', '九等', '十等']:
        if grade not in density_stats.columns:
            density_stats[grade] = 0
    
    # 添加合计行
    density_stats.loc['合计'] = density_stats.sum()
    
    # 按照图片中的顺序排列容重区间
    density_order = ['<4.50', '4.50-5.50', '5.50-6.50', '6.50-7.50', '>7.50', '合计']
    
    # 重新整理结果DataFrame以匹配所需格式
    result_df = pd.DataFrame()
    result_df['酸碱度'] = density_order
    
    # 设置结果DataFrame的索引以便后续操作
    result_df = result_df.set_index('酸碱度')
    
    # 对于每个等级，添加面积和比例列
    for grade in ['一等', '二等', '三等', '四等', '五等', '六等', '七等', '八等', '九等', '十等']:
        # 为不存在的容重区间填充0
        for density in density_order:
            if density != '合计' and density not in density_stats.index:
                density_stats.loc[density, grade] = 0
        
        # 计算每个酸碱度区间在该等级中的比例
        total_grade_area = density_stats.loc['合计', grade] if '合计' in density_stats.index else density_stats[grade].sum()
        
        if total_grade_area > 0:
            for density in density_order:
                if density != '合计':
                    # 如果酸碱度区间在统计数据中不存在，则面积为0
                    area = density_stats.loc[density, grade] if density in density_stats.index else 0
                    ratio = (area / total_grade_area * 100).round(2) if total_grade_area > 0 else 0
                else:
                    # 合计行的数据
                    area = total_grade_area
                    ratio = 100.0
                
                # 添加到结果DataFrame
                result_df.loc[density, f'{grade}_面积/亩'] = area
                result_df.loc[density, f'{grade}_比例/%'] = ratio
    
    # 重置索引并返回结果
    result_df = result_df.reset_index()
    return result_df

# 各等级耕地土壤有机质含量对比
def table_organic_matter(df,area_columns='project_Area',level_columns='综合质量等级',density_column='有机质'):
    # 将面积单位(平方米)换算为亩
    calc_df = df.copy()
    calc_df[area_columns] = calc_df[area_columns] * 0.0015
    
    # 将有机质分段
    bins = [0, 10, 15, 20, 25, 30, float('inf')]
    labels = ['<10', '10-15', '15-20', '20-25', '25-30', '>=30']
    calc_df['有机质区间'] = pd.cut(calc_df[density_column], bins=bins, labels=labels, right=False)
    
    # 按有机质区间和等级分组统计面积
    density_stats = calc_df.groupby(['有机质区间', level_columns])[area_columns].sum().unstack(fill_value=0)
    
    # 确保所有等级列存在（一等到十等）
    for grade in ['一等', '二等', '三等', '四等', '五等', '六等', '七等', '八等', '九等', '十等']:
        if grade not in density_stats.columns:
            density_stats[grade] = 0
    
    # 添加合计行
    density_stats.loc['合计'] = density_stats.sum()
    
    # 按照图片中的顺序排列有机质区间
    density_order = ['<10', '10-15', '15-20', '20-25', '25-30', '>=30', '合计']
    
    # 重新整理结果DataFrame以匹配所需格式
    result_df = pd.DataFrame()
    result_df['有机质'] = density_order
    
    # 设置结果DataFrame的索引以便后续操作
    result_df = result_df.set_index('有机质')
    
    # 对于每个等级，添加面积和比例列
    for grade in ['一等', '二等', '三等', '四等', '五等', '六等', '七等', '八等', '九等', '十等']:
        # 为不存在的容重区间填充0
        for density in density_order:
            if density != '合计' and density not in density_stats.index:
                density_stats.loc[density, grade] = 0
        
        # 计算每个有机质区间在该等级中的比例
        total_grade_area = density_stats.loc['合计', grade] if '合计' in density_stats.index else density_stats[grade].sum()
        
        if total_grade_area > 0:
            for density in density_order:
                if density != '合计':
                    # 如果有机质区间在统计数据中不存在，则面积为0
                    area = density_stats.loc[density, grade] if density in density_stats.index else 0
                    ratio = (area / total_grade_area * 100).round(2) if total_grade_area > 0 else 0
                else:
                    # 合计行的数据
                    area = total_grade_area
                    ratio = 100.0
                
                # 添加到结果DataFrame
                result_df.loc[density, f'{grade}_面积/亩'] = area
                result_df.loc[density, f'{grade}_比例/%'] = ratio
    
    # 重置索引并返回结果
    result_df = result_df.reset_index()
    return result_df

# 各等级耕地土壤有效磷含量对比
def table_phosphorus(df,area_columns='project_Area',level_columns='综合质量等级',density_column='有效磷'):
    # 将面积单位(平方米)换算为亩
    calc_df = df.copy()
    calc_df[area_columns] = calc_df[area_columns] * 0.0015
    
    # 将有效磷分段
    bins = [0, 5, 10, 20, 30, float('inf')]
    labels = ['<5', '5-10', '10-20', '20-30', '>30']
    calc_df['有效磷区间'] = pd.cut(calc_df[density_column], bins=bins, labels=labels, right=False)
    
    # 按有效磷区间和等级分组统计面积
    density_stats = calc_df.groupby(['有效磷区间', level_columns])[area_columns].sum().unstack(fill_value=0)
    
    # 确保所有等级列存在（一等到十等）
    for grade in ['一等', '二等', '三等', '四等', '五等', '六等', '七等', '八等', '九等', '十等']:
        if grade not in density_stats.columns:
            density_stats[grade] = 0
    
    # 添加合计行
    density_stats.loc['合计'] = density_stats.sum()
    
    # 按照图片中的顺序排列容重区间
    density_order = ['<5', '5-10', '10-20', '20-30', '>30', '合计']
    
    # 重新整理结果DataFrame以匹配所需格式
    result_df = pd.DataFrame()
    result_df['有效磷'] = density_order
    
    # 设置结果DataFrame的索引以便后续操作
    result_df = result_df.set_index('有效磷')
    
    # 对于每个等级，添加面积和比例列
    for grade in ['一等', '二等', '三等', '四等', '五等', '六等', '七等', '八等', '九等', '十等']:
        # 为不存在的容重区间填充0
        for density in density_order:
            if density != '合计' and density not in density_stats.index:
                density_stats.loc[density, grade] = 0
        
        # 计算每个有效磷区间在该等级中的比例
        total_grade_area = density_stats.loc['合计', grade] if '合计' in density_stats.index else density_stats[grade].sum()
        
        if total_grade_area > 0:
            for density in density_order:
                if density != '合计':
                    # 如果有效磷区间在统计数据中不存在，则面积为0
                    area = density_stats.loc[density, grade] if density in density_stats.index else 0
                    ratio = (area / total_grade_area * 100).round(2) if total_grade_area > 0 else 0
                else:
                    # 合计行的数据
                    area = total_grade_area
                    ratio = 100.0
                
                # 添加到结果DataFrame
                result_df.loc[density, f'{grade}_面积/亩'] = area
                result_df.loc[density, f'{grade}_比例/%'] = ratio
    
    # 重置索引并返回结果
    result_df = result_df.reset_index()
    return result_df

# 各等级耕地土壤速效钾含量对比
def table_potassium(df,area_columns='project_Area',level_columns='综合质量等级',density_column='速效钾'):
    # 将面积单位(平方米)换算为亩
    calc_df = df.copy()
    calc_df[area_columns] = calc_df[area_columns] * 0.0015
    
    # 将速效钾分段
    bins = [0, 50, 100, 150, 200, float('inf')]
    labels = ['<=50', '50-100', '100-150', '150-200', '>200']
    calc_df['速效钾区间'] = pd.cut(calc_df[density_column], bins=bins, labels=labels, right=False)
    
    # 按速效钾区间和等级分组统计面积
    density_stats = calc_df.groupby(['速效钾区间', level_columns])[area_columns].sum().unstack(fill_value=0)
    
    # 确保所有等级列存在（一等到十等）
    for grade in ['一等', '二等', '三等', '四等', '五等', '六等', '七等', '八等', '九等', '十等']:
        if grade not in density_stats.columns:
            density_stats[grade] = 0
    
    # 添加合计行
    density_stats.loc['合计'] = density_stats.sum()
    
    # 按照图片中的顺序排列速效钾区间
    density_order = ['<=50', '50-100', '100-150', '150-200', '>200', '合计']
    
    # 重新整理结果DataFrame以匹配所需格式
    result_df = pd.DataFrame()
    result_df['速效钾'] = density_order
    
    # 设置结果DataFrame的索引以便后续操作
    result_df = result_df.set_index('速效钾')
    
    # 对于每个等级，添加面积和比例列
    for grade in ['一等', '二等', '三等', '四等', '五等', '六等', '七等', '八等', '九等', '十等']:
        # 为不存在的速效钾区间填充0
        for density in density_order:
            if density != '合计' and density not in density_stats.index:
                density_stats.loc[density, grade] = 0
        
        # 计算每个速效钾区间在该等级中的比例
        total_grade_area = density_stats.loc['合计', grade] if '合计' in density_stats.index else density_stats[grade].sum()
        
        if total_grade_area > 0:
            for density in density_order:
                if density != '合计':
                    # 如果速效钾区间在统计数据中不存在，则面积为0
                    area = density_stats.loc[density, grade] if density in density_stats.index else 0
                    ratio = (area / total_grade_area * 100).round(2) if total_grade_area > 0 else 0
                else:
                    # 合计行的数据
                    area = total_grade_area
                    ratio = 100.0
                
                # 添加到结果DataFrame
                result_df.loc[density, f'{grade}_面积/亩'] = area
                result_df.loc[density, f'{grade}_比例/%'] = ratio
    
    # 重置索引并返回结果
    result_df = result_df.reset_index()
    return result_df

# 各等级耕地灌溉能力对比
def table_irrigation(df,area_columns='project_Area',level_columns='综合质量等级',texture_column='灌溉能力'):
    # 将面积单位(平方米)换算为亩
    calc_df = df.copy()
    calc_df[area_columns] = calc_df[area_columns] * 0.0015
    
    # 按耕层质地和等级分组统计面积
    texture_stats = calc_df.groupby([texture_column, level_columns])[area_columns].sum().unstack(fill_value=0)
    
    # 确保所有等级列存在（一等到十等）
    for grade in ['一等', '二等', '三等', '四等', '五等', '六等', '七等', '八等', '九等', '十等']:
        if grade not in texture_stats.columns:
            texture_stats[grade] = 0
    
    # 添加合计行
    texture_stats.loc['合计'] = texture_stats.sum()
    
    # 按照图片中的顺序排列灌溉能力类型
    texture_order = [
        '充分满足', 
        '满足', 
        '基本满足', 
        '不满足', 
        '合计'
    ]
    
    # 重新整理结果DataFrame以匹配所需格式
    result_df = pd.DataFrame()
    result_df['灌溉能力'] = texture_order
    
    # 设置结果DataFrame的索引以便后续操作
    result_df = result_df.set_index('灌溉能力')
    
    # 对于每个等级，添加面积和比例列
    for grade in ['一等', '二等', '三等', '四等', '五等', '六等', '七等', '八等', '九等', '十等']:
        # 为不存在的灌溉能力类型填充0
        for texture in texture_order:
            if texture != '合计' and texture not in texture_stats.index:
                texture_stats.loc[texture, grade] = 0
        
        # 计算每个灌溉能力类型在该等级中的比例
        total_grade_area = texture_stats.loc['合计', grade] if '合计' in texture_stats.index else texture_stats[grade].sum()
        
        if total_grade_area > 0:
            for texture in texture_order:
                if texture != '合计':
                    # 如果灌溉能力类型在统计数据中不存在，则面积为0
                    area = texture_stats.loc[texture, grade] if texture in texture_stats.index else 0
                    ratio = (area / total_grade_area * 100).round(2) if total_grade_area > 0 else 0
                else:
                    # 合计行的数据
                    area = total_grade_area
                    ratio = 100.0
                
                # 添加到结果DataFrame
                result_df.loc[texture, f'{grade}_面积/亩'] = area
                result_df.loc[texture, f'{grade}_比例/%'] = ratio
    
    # 重置索引并返回结果
    result_df = result_df.reset_index()
    return result_df

# 各等级耕地排水能力对比
def table_drainage(df,area_columns='project_Area',level_columns='综合质量等级',texture_column='排水能力'):
    # 将面积单位(平方米)换算为亩
    calc_df = df.copy()
    calc_df[area_columns] = calc_df[area_columns] * 0.0015
    
    # 按排水能力类型和等级分组统计面积
    texture_stats = calc_df.groupby([texture_column, level_columns])[area_columns].sum().unstack(fill_value=0)
    
    # 确保所有等级列存在（一等到十等）
    for grade in ['一等', '二等', '三等', '四等', '五等', '六等', '七等', '八等', '九等', '十等']:
        if grade not in texture_stats.columns:
            texture_stats[grade] = 0
    
    # 添加合计行
    texture_stats.loc['合计'] = texture_stats.sum()
    
    # 按照图片中的顺序排列耕层质地类型
    texture_order = [
        '充分满足', 
        '满足', 
        '基本满足', 
        '不满足', 
        '合计'
    ]
    
    # 重新整理结果DataFrame以匹配所需格式
    result_df = pd.DataFrame()
    result_df['排水能力'] = texture_order
    
    # 设置结果DataFrame的索引以便后续操作
    result_df = result_df.set_index('排水能力')
    
    # 对于每个等级，添加面积和比例列
    for grade in ['一等', '二等', '三等', '四等', '五等', '六等', '七等', '八等', '九等', '十等']:
        # 为不存在的排水能力类型填充0
        for texture in texture_order:
            if texture != '合计' and texture not in texture_stats.index:
                texture_stats.loc[texture, grade] = 0
        
        # 计算每个排水能力类型在该等级中的比例
        total_grade_area = texture_stats.loc['合计', grade] if '合计' in texture_stats.index else texture_stats[grade].sum()
        
        if total_grade_area > 0:
            for texture in texture_order:
                if texture != '合计':
                    # 如果排水能力类型在统计数据中不存在，则面积为0
                    area = texture_stats.loc[texture, grade] if texture in texture_stats.index else 0
                    ratio = (area / total_grade_area * 100).round(2) if total_grade_area > 0 else 0
                else:
                    # 合计行的数据
                    area = total_grade_area
                    ratio = 100.0
                
                # 添加到结果DataFrame
                result_df.loc[texture, f'{grade}_面积/亩'] = area
                result_df.loc[texture, f'{grade}_比例/%'] = ratio
    
    # 重置索引并返回结果
    result_df = result_df.reset_index()
    return result_df

In [3]:
# 数据路径
data_path = r"G:\soil_property_result\qzs\grade_evaluation\result\grade_evaluation_result_have_channeng.xlsx"

In [4]:
df = pd.read_excel(data_path)

In [5]:
# 筛选仅为清镇市的耕地（XZQMC不为空）
df = df[~df['XZQMC'].isnull()]

In [6]:
df['平差面积'].sum()*0.0015

601745.1564373402

In [7]:

# 表7.6 耕地质量等级总体状况
df_gdzldjzk= table_7_6(df,area_columns='平差面积',level_columns='综合质量等级',capacity_columns='预测产能')
# 表7.7 乡镇耕地质量等级分布
df_xzgdzldjqk= table_7_7(df,township_columns='XZQMC',level_columns='综合质量等级',area_columns='平差面积')
# 表7.8 土壤类型耕地质量等级状况
df_trlxgdzldjqk = table_7_8(df,area_columns='平差面积',level_columns='综合质量等级',type_column='Class1_tl',sub_column='Class1_yl',cate_column='Class1_ts',spe_column='Class1')
# 表7.9 分布特征
df_1djfbtz = table_7_9_distribution(df,area_columns='平差面积',level_columns='综合质量等级',township_columns='XZQMC',level_value='一等')
df_2djfbtz = table_7_9_distribution(df,area_columns='平差面积',level_columns='综合质量等级',township_columns='XZQMC',level_value='二等')
df_3djfbtz = table_7_9_distribution(df,area_columns='平差面积',level_columns='综合质量等级',township_columns='XZQMC',level_value='三等')
df_4djfbtz = table_7_9_distribution(df,area_columns='平差面积',level_columns='综合质量等级',township_columns='XZQMC',level_value='四等')
df_5djfbtz = table_7_9_distribution(df,area_columns='平差面积',level_columns='综合质量等级',township_columns='XZQMC',level_value='五等')
df_6djfbtz = table_7_9_distribution(df,area_columns='平差面积',level_columns='综合质量等级',township_columns='XZQMC',level_value='六等')
df_7djfbtz = table_7_9_distribution(df,area_columns='平差面积',level_columns='综合质量等级',township_columns='XZQMC',level_value='七等')
df_8djfbtz = table_7_9_distribution(df,area_columns='平差面积',level_columns='综合质量等级',township_columns='XZQMC',level_value='八等')
df_9djfbtz = table_7_9_distribution(df,area_columns='平差面积',level_columns='综合质量等级',township_columns='XZQMC',level_value='九等')
df_10djfbtz = table_7_9_distribution(df,area_columns='平差面积',level_columns='综合质量等级',township_columns='XZQMC',level_value='十等')
# 表7.9 性状特征
df_1djxztz = table_7_9_properties(df,area_columns='平差面积',level_columns='综合质量等级',type_column='Class1_tl',sub_column='Class1_yl',cate_column='Class1_ts',spe_column='Class1',level_value='一等',capacity_columns='预测产能')
df_2djxztz = table_7_9_properties(df,area_columns='平差面积',level_columns='综合质量等级',type_column='Class1_tl',sub_column='Class1_yl',cate_column='Class1_ts',spe_column='Class1',level_value='二等',capacity_columns='预测产能')
df_3djxztz = table_7_9_properties(df,area_columns='平差面积',level_columns='综合质量等级',type_column='Class1_tl',sub_column='Class1_yl',cate_column='Class1_ts',spe_column='Class1',level_value='三等',capacity_columns='预测产能')
df_4djxztz = table_7_9_properties(df,area_columns='平差面积',level_columns='综合质量等级',type_column='Class1_tl',sub_column='Class1_yl',cate_column='Class1_ts',spe_column='Class1',level_value='四等',capacity_columns='预测产能')
df_5djxztz = table_7_9_properties(df,area_columns='平差面积',level_columns='综合质量等级',type_column='Class1_tl',sub_column='Class1_yl',cate_column='Class1_ts',spe_column='Class1',level_value='五等',capacity_columns='预测产能')
df_6djxztz = table_7_9_properties(df,area_columns='平差面积',level_columns='综合质量等级',type_column='Class1_tl',sub_column='Class1_yl',cate_column='Class1_ts',spe_column='Class1',level_value='六等',capacity_columns='预测产能')
df_7djxztz = table_7_9_properties(df,area_columns='平差面积',level_columns='综合质量等级',type_column='Class1_tl',sub_column='Class1_yl',cate_column='Class1_ts',spe_column='Class1',level_value='七等',capacity_columns='预测产能')
df_8djxztz = table_7_9_properties(df,area_columns='平差面积',level_columns='综合质量等级',type_column='Class1_tl',sub_column='Class1_yl',cate_column='Class1_ts',spe_column='Class1',level_value='八等',capacity_columns='预测产能')
df_9djxztz = table_7_9_properties(df,area_columns='平差面积',level_columns='综合质量等级',type_column='Class1_tl',sub_column='Class1_yl',cate_column='Class1_ts',spe_column='Class1',level_value='九等',capacity_columns='预测产能')
df_10djxztz = table_7_9_properties(df,area_columns='平差面积',level_columns='综合质量等级',type_column='Class1_tl',sub_column='Class1_yl',cate_column='Class1_ts',spe_column='Class1',level_value='十等',capacity_columns='预测产能')
# 地形部位对比
df_dxbwdb = table_terrain(df,area_columns='平差面积',level_columns='综合质量等级',terrain_column='地形部位')
# 有效土层厚度对比
df_yxtcdb = table_thickness(df,area_columns='平差面积',level_columns='综合质量等级',thickness_column='有效土层厚度')
# 质地构型
df_zdgx = table_texture_config(df,area_columns='平差面积',level_columns='综合质量等级',texture_column='质地构型')
# 海拔高度
df_hbgd = table_elevation(df,area_columns='平差面积',level_columns='综合质量等级',elevation_column='海拔高度')
# 耕层厚度
df_gchd = table_layer_thickness(df,area_columns='平差面积',level_columns='综合质量等级',thickness_column='耕层厚度')
# 耕层质地
df_gczd = table_layer_texture(df,area_columns='平差面积',level_columns='综合质量等级',texture_column='耕层质地')
# 土壤容重
df_trrz = table_bulk_density(df,area_columns='平差面积',level_columns='综合质量等级',density_column='土壤容重')
# 土壤酸碱度
df_trsd = table_ph(df,area_columns='平差面积',level_columns='综合质量等级',density_column='酸碱度')
# 土壤有机质
df_yzz = table_organic_matter(df,area_columns='平差面积',level_columns='综合质量等级',density_column='有机质')
# 有效磷
df_yxl = table_phosphorus(df,area_columns='平差面积',level_columns='综合质量等级',density_column='有效磷')
# 速效钾
df_sxj = table_potassium(df,area_columns='平差面积',level_columns='综合质量等级',density_column='速效钾')
# 灌溉能力
df_ggnl = table_irrigation(df,area_columns='平差面积',level_columns='综合质量等级',texture_column='灌溉能力')
# 排水能力
df_psnl = table_drainage(df,area_columns='平差面积',level_columns='综合质量等级',texture_column='排水能力')


In [8]:
# 将所有表写入Excel的不同sheet
tables = {
    '总体状况': df_gdzldjzk,
    '乡镇质量': df_xzgdzldjqk,
    '土壤类型': df_trlxgdzldjqk,
    '一等分布': df_1djfbtz,
    '一等性状': df_1djxztz,
    '二等分布': df_2djfbtz,
    '二等性状': df_2djxztz,
    '三等分布': df_3djfbtz,
    '三等性状': df_3djxztz,
    '四等分布': df_4djfbtz,
    '四等性状': df_4djxztz,
    '五等分布': df_5djfbtz,
    '五等性状': df_5djxztz,
    '六等分布': df_6djfbtz,
    '六等性状': df_6djxztz,
    '七等分布': df_7djfbtz,
    '七等性状': df_7djxztz,
    '八等分布': df_8djfbtz,
    '八等性状': df_8djxztz,
    '九等分布': df_9djfbtz,
    '九等性状': df_9djxztz,
    '十等分布': df_10djfbtz,
    '十等性状': df_10djxztz,
    '地形部位对比': df_dxbwdb,
    '有效土层厚度对比': df_yxtcdb,
    '质地构型对比': df_zdgx,
    '海拔高度对比': df_hbgd,
    '耕层厚度对比': df_gchd,
    '耕层质地对比': df_gczd,
    '土壤容重对比': df_trrz,
    '土壤酸碱度对比': df_trsd,
    '有机质含量对比': df_yzz,
    '有效磷含量对比': df_yxl,
    '速效钾含量对比': df_sxj,
    '灌溉能力对比': df_ggnl,
    '排水能力对比': df_psnl
}

save_path = r"G:\soil_property_result\qzs\grade_evaluation\result\report"
os.makedirs(save_path, exist_ok=True)

# 使用with语句自动管理资源关闭
with pd.ExcelWriter(os.path.join(save_path, '耕地质量等级统计表.xlsx')) as excel_writer:

    for table_name, table_data in tables.items():
        try:
            table_data.to_excel(excel_writer, sheet_name=table_name, index=False)
        except Exception as e:
            print(f"写入表格{table_name}时出现错误: {e}")
            continue

In [9]:
df['质地构型'].value_counts()

质地构型
上松下紧型    40086
海绵型      31403
紧实型      13961
松散型        621
上紧下松型       24
Name: count, dtype: int64

In [11]:
df['耕层质地'].value_counts()


耕层质地
壤质黏土        45010
黏土          25577
黏壤土          4265
粉(砂)质黏土      3760
砂质黏土         1911
重黏土          1797
砂质黏壤土        1514
粉(砂)质黏壤土     1075
砂质壤土          851
粉(砂)质壤土       243
壤土             92
Name: count, dtype: int64